In [13]:
%reload_ext autoreload
%autoreload 2

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

from sklearn.preprocessing import MinMaxScaler

# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [14]:
N_FOLD = 5
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = '601-NN'
DATA_NAME = '601'
mkdir(f'./models/{SOL_NAME}/')

In [15]:
# CONSTANT
MEAN = -5.762330803300896
STD = 0.6339307835941186
EPS = 1e-9

In [16]:
# get ids
list_stock_id = get_stock_id()
list_time_id = get_time_id()

# Functions

In [17]:
def transform_target(target):
    return (np.log(target + EPS) - MEAN) / STD

def inverse_target(target):
    return np.exp(MEAN + STD * target) - EPS

def np_rmspe(y_true, y_pred):
    y_true = inverse_target(y_true)
    y_pred = inverse_target(y_pred)
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def mspe_loss(y_true, y_pred):
    y_true = K.exp(MEAN + STD * y_true) - EPS
    y_pred = K.exp(MEAN + STD * y_pred) - EPS
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

In [18]:
# def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
#     inp = tf.keras.layers.Input(shape=(num_columns,))
#     x = tf.keras.layers.BatchNormalization()(inp)
#     for i in range(len(hidden_units)):
#         x = tf.keras.layers.Dense(hidden_units[i])(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
#     # Output layer
#     x = tf.keras.layers.Dense(num_labels)(x)
#     out = tf.keras.layers.Activation('linear')(x)
#     model = tf.keras.models.Model(inputs=inp, outputs=out)
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#         loss=[mspe_loss],
#     )
#     return model

In [19]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    stock_inp = tf.keras.layers.Input(shape=(1,))
    stock_embedded = tf.keras.layers.Embedding(127, 24, input_length=1, name='stock_embedding')(stock_inp)
    stock_flattened = tf.keras.layers.Flatten()(stock_embedded)
    x = tf.keras.layers.Concatenate()([stock_flattened, inp])
    x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        # x = tf.keras.layers.Dense(hidden_units[i], kernel_regularizer=tf.keras.regularizers.l1(dropout_rates))(x)
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        # x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)
    # Output layer
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('linear')(x)
    model = tf.keras.models.Model(inputs=[stock_inp, inp], outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=mspe_loss,
    )
    return model

# Loading data

In [20]:
# train
df_train = dt.fread(f'./dataset/public_train_{DATA_NAME}_NN.csv').to_pandas()
fea_cols = [f for f in df_train if f not in ['time_id', 'target', 'pred_NN', 'stock_id', 'row_id']]

# result
df_result = dt.fread('./dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [21]:
df_train['target'] = transform_target(df_train['target'])
df_train = gen_row_id(df_train)
df_train = add_time_fold(df_train, N_FOLD)

# Evaluation

In [22]:
batch_size = 1024
# hidden_units = [256]*4
hidden_units = [64, 32, 16]
dropout_rates = 0
learning_rate = 6e-3
epochs = 1000

list_rmspe = [1 for _ in range(N_FOLD)]

In [25]:
n_trials = 3
for _ in range(n_trials):
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]

        S_train = df_tr['stock_id'].values
        X_train = df_tr[fea_cols].values
        y_train = df_tr[['target']].values
        S_test = df_te['stock_id'].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['target']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)

        scaler = MinMaxScaler(feature_range=(-1, 1))         
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        save_pickle(scaler, f'./models/{SOL_NAME}/minmax_scaler_{i_fold}.pkl')

        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_601_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=28, restore_best_weights=True, verbose=2)

        model = create_mlp(X_train.shape[1], 1, hidden_units, dropout_rates, learning_rate) 
        history = model.fit([S_train, X_train], y_train, 
            epochs=epochs,
            validation_data=([S_test, X_test], y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        )
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict([S_test, X_test], batch_size=len(y_test))
        curr_rmspe = np_rmspe(y_test, y_pred)
        if curr_rmspe < list_rmspe[i_fold]:
            # model.save(ckp_path)
            list_rmspe[i_fold] = curr_rmspe
            # generate and save preds
            df_result.loc[idx_test, 'pred'] = inverse_target(y_pred)
        clear_output()
        print(list_rmspe)
        calc_metric(df_result.fillna(0))
    break

[0.2096668020547297, 0.21244677695053857, 0.21349347770451443, 0.21000272941228676, 0.21920029661194654]
   R2: 0.8393
RMSPE: 0.2130


In [24]:
# df_result.to_csv(f'./results/{SOL_NAME}.csv', index=False)